In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu, softmax

In [ ]:
training_file_path = '/content/drive/MyDrive/Colab Notebooks/training.csv'
training_data = pd.read_csv(training_file_path) 
training_data.columns

test_file_path = '/content/drive/MyDrive/Colab Notebooks/testing.csv'
test_data = pd.read_csv(test_file_path) 
test_data.columns

In [ ]:
# Extract the training data
X_trains = training_data.iloc[:, 1:]
y_trains = training_data.iloc[:, 0]

# Print the first few rows of the 'X' data
print(X_trains.head())
# Print the first few values of the 'y' data
print(y_trains.head())

# Extract the test data
X_tests = test_data.iloc[:, 1:]
y_tests = test_data.iloc[:, 0]

# Print the first few rows of the 'X' data
print(X_tests.head())
# Print the first few values of the 'y' data
print(y_tests.head())

In [ ]:
# Convert X
X_trains = np.array(X_trains)
X_tests = np.array(X_tests)

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to the 'y' data
label_encoder.fit(y_trains)
label_encoder.fit(y_tests)

# Transform the 'y' data into encoded values
y_trains= label_encoder.transform(y_trains)
y_tests = label_encoder.transform(y_tests)

In [ ]:
#  Oversample the minority classes using SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(sampling_strategy='auto', random_state=42)

#  Undersample the majority class using RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

#  Apply SMOTE to oversample the minority classes in the training data
X_trains, y_trains = smote.fit_resample(X_trains, y_trains)

#  Apply RandomUnderSampler to undersample the majority class in the training data
X_trains, y_trains = rus.fit_resample(X_trains, y_trains)